# Mapping Agricultural-Driven Deforestation in Central Kalimantan with Remote Sensing

Lu Yii Wong 

May 2025

## Part 1: Setup and Data Aquisition 

In [1]:
# Setup Libraries
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import geopandas as gpd

In [2]:
# Authenticate GEE
ee.Authenticate()
ee.Initialize(project='ee-wongluyii')


Successfully saved authorization token.


In [3]:
# Load Boundary for Area of Interest
# Load shapefile
gdf = gpd.read_file("data/Central_Kalimantan.geojson").to_crs(epsg=4326)

# Convert to Earth Engine object
roi = geemap.geopandas_to_ee(gdf)

In [9]:
# Define years of interest
years = [2018, 2019, 2020, 2021, 2022] 

In [5]:
# Create Functions to pull data following the given parameters
# Visualization parameters
s2_vis = {
    "bands": ["B4", "B3", "B2"],  # True color
    "min": 0,
    "max": 0.3
}

# Cloud masking function
def maskS2clouds(image):
    qa = image.select("QA60")
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Create map
Map = geemap.Map()
Map.centerObject(roi, 7)

In [11]:
# Create map
Map = geemap.Map()
Map.centerObject(roi, 7)

# Load S2_SR and S2_CLOUD_PROBABILITY as a joined collection
s2_sr = ee.ImageCollection("COPERNICUS/S2_SR")
s2_clouds = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")

# Join function
def join_collections(sr_col, cloud_col):
    join = ee.Join.saveFirst("cloud_mask")
    filter_time_eq = ee.Filter.equals(leftField="system:index", rightField="system:index")
    joined = join.apply(sr_col, cloud_col, filter_time_eq)
    return ee.ImageCollection(joined)

def maskClouds(image):
    cloud_mask = ee.Image(image.get("cloud_mask")).select("probability")
    mask = cloud_mask.lt(40)  # mask where cloud probability < 40%
    return image.updateMask(mask).divide(10000)


# Loop
for year in years:
    start = f"{year}-01-01"
    end = f"{year}-12-31"

    s2 = s2_sr.filterDate(start, end).filterBounds(roi)
    s2_prob = s2_clouds.filterDate(start, end).filterBounds(roi)

    s2_joined = join_collections(s2, s2_prob).map(maskClouds)

    s2_median = s2_joined.median().clip(roi)

    Map.addLayer(s2_median, s2_vis, f"Sentinel-2 {year}")



# Add ROI for reference
Map.addLayer(roi, {}, "Study Area")
Map

Map(center=[-1.6022213970366683, 113.41802211199322], controls=(WidgetControl(options=['position', 'transparen…

In [13]:
# Define years of interest
years = [2017, 2020, 2023] 

# Create map
Map = geemap.Map()
Map.centerObject(roi, 7)

# Load S2_SR and S2_CLOUD_PROBABILITY as a joined collection
s2_sr = ee.ImageCollection("COPERNICUS/S2_SR")
s2_clouds = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")

# Join function
def join_collections(sr_col, cloud_col):
    join = ee.Join.saveFirst("cloud_mask")
    filter_time_eq = ee.Filter.equals(leftField="system:index", rightField="system:index")
    joined = join.apply(sr_col, cloud_col, filter_time_eq)
    return ee.ImageCollection(joined)

def maskClouds(image):
    cloud_mask = ee.Image(image.get("cloud_mask")).select("probability")
    mask = cloud_mask.lt(40)  # mask where cloud probability < 40%
    return image.updateMask(mask).divide(10000)


# Loop
for year in years:
    start = f"{year}-01-01"
    end = f"{year}-12-31"

    s2 = s2_sr.filterDate(start, end).filterBounds(roi)
    s2_prob = s2_clouds.filterDate(start, end).filterBounds(roi)

    s2_joined = join_collections(s2, s2_prob).map(maskClouds)

    s2_median = s2_joined.median().clip(roi)

    Map.addLayer(s2_median, s2_vis, f"Sentinel-2 {year}")



# Add ROI for reference
Map.addLayer(roi, {}, "Study Area")
Map

Map(center=[-1.6022213970366683, 113.41802211199322], controls=(WidgetControl(options=['position', 'transparen…

Only images from 2020 is being pulled correctly.